In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [8]:
# Load the data

application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")


In [9]:
# Data Preprocessing
application_df = application_df.drop(columns=["EIN", "NAME", "SPECIAL_CONSIDERATIONS"])
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [15]:
application_df = pd.get_dummies(application_df)
X = application_df.drop(columns="IS_SUCCESSFUL").values
y = application_df["IS_SUCCESSFUL"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [16]:
# Design
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  100  #neurons
hidden_nodes_layer2 = 50   #neurons
hidden_nodes_layer3 = 20   #layer

nn_optimized = tf.keras.models.Sequential()

# Regularization strength
l2_strength = 0.001

# First layer
nn_optimized.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_strength))
)

# Second layer
nn_optimized.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_strength)))

# Third layer
nn_optimized.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(l2_strength)))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_optimized.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [17]:
# Adding early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [18]:
# Training the optimized model
fit_model_optimized = nn_optimized.fit(X_train_scaled, y_train, epochs=150)  #adjust epochs


Epoch 1/150
804/804 [==============================] - 6s 4ms/step - loss: 0.6678 - accuracy: 0.7238
Epoch 2/150
804/804 [==============================] - 4s 5ms/step - loss: 0.6024 - accuracy: 0.7302
Epoch 3/150
804/804 [==============================] - 5s 7ms/step - loss: 0.5810 - accuracy: 0.7325
Epoch 4/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5701 - accuracy: 0.7338
Epoch 5/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5657 - accuracy: 0.7330
Epoch 6/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5624 - accuracy: 0.7334
Epoch 7/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5605 - accuracy: 0.7338
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5598 - accuracy: 0.7341
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7352
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5584 - accura

In [19]:
model_loss_optimized, model_accuracy_optimized = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss_optimized}, Accuracy: {model_accuracy_optimized}")

268/268 - 1s - loss: 0.5602 - accuracy: 0.7264 - 772ms/epoch - 3ms/step
Loss: 0.5601846575737, Accuracy: 0.7264139652252197


In [ ]:
# Save the optimized model
nn_optimized.save("AlphabetSoupCharity_Optimization.h5")
